# Quotebank Exploratory Data Analysis

TODO add description

## Dataset loading

In [ ]:
import os
import json
import csv
import tld
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd

client = Client(n_workers=2, threads_per_worker=2, memory_limit='6GB')

In [ ]:
DATASET_PATH = "datasets"

filename = "quotes-2020.parquet"

dataset = dd.read_parquet(os.path.join(DATASET_PATH, filename))

In [ ]:
# Convert date column to datetime objects
dataset["date"] = dd.to_datetime(dataset["date"])

In [ ]:
# Print the dataset
dataset.head(10)

## Number of quotes per day

In [ ]:
ds_by_day = dataset.set_index("date").resample("1D")
number_of_quotes_per_day = ds_by_day.size().compute()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.set_title("Number of Quotes per Day")

# Major ticks every week.
fmt_week = mdates.WeekdayLocator(interval=1)
ax.xaxis.set_major_locator(fmt_week)

# Minor ticks every day.
fmt_day = mdates.DayLocator(interval=1)
ax.xaxis.set_minor_locator(fmt_day)

# Text in the x axis will be displayed in 'YYYY-mm-dd' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))

# Round to nearest months.
datemin = np.datetime64(number_of_quotes_per_day.index[0], 'W')
datemax = np.datetime64(number_of_quotes_per_day.index[-1], 'W') + np.timedelta64(1, 'W')
ax.set_xlim(datemin, datemax)
ax.grid(True)
ax.plot(number_of_quotes_per_day, c="r", linewidth=1, alpha=0.75, marker="o", linestyle="--", markersize=5)

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()

Why is there a sharp decrease in the middle of each week. Is it because of the weekdays/weekends?

In [ ]:
dataset["day_of_week"] = dataset["date"].dt.day_name()
ds_by_weekdays = dataset.groupby(dataset["day_of_week"])
number_of_quotes_per_weekday = ds_by_weekdays.size().compute()

In [ ]:
# Sort the index by the day of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_type = (days_of_week[:5], days_of_week[5:])

mapping = {day: i for i, day in enumerate(reversed(days_of_week))}
key = number_of_quotes_per_weekday.index.map(mapping)
number_of_quotes_per_weekday = number_of_quotes_per_weekday.iloc[key.argsort()]

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].set_title("Number of Quotes per Weekday", fontsize=12)
ax[1].set_title("Percent of Quotes per Day Type", fontsize=12)
fig.suptitle("Day of the Week Analysis", fontsize=16)

colors = ["#7DBBC3", "#7DBBC3", "#F4B9B2", "#F4B9B2", "#F4B9B2", "#F4B9B2", "#F4B9B2"]
ax[0].barh(number_of_quotes_per_weekday.index, number_of_quotes_per_weekday, edgecolor="black", color=colors)

ax[1].pie([number_of_quotes_per_weekday.loc[x].mean() for x in day_type], labels=["Weekdays", "Weekends"], autopct='%.1f%%', shadow=True, startangle=90, colors=["#F4B9B2", "#7DBBC3"], wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'}, explode=(0, 0.1))
plt.tight_layout()

Aha, we were right! Apparently many people decide to stay silent during the weekend.

Let's explore the trends

In [ ]:
quote_trends_daily = pd.DataFrame({"n_quotes": number_of_quotes_per_day}, index=number_of_quotes_per_day.index)
quote_trends_7 = quote_trends_daily["n_quotes"].rolling(window=7, center=True, min_periods=5).mean()
quote_trends_30 = quote_trends_daily["n_quotes"].rolling(window=30, center=True, min_periods=20).mean()

In [ ]:
# Plot daily, 7-day rolling mean, and 30-day rolling mean
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.plot(number_of_quotes_per_day, marker=".", markersize=4.5, color="0.45", linestyle="dotted", label="Daily")
ax.plot(quote_trends_7, linewidth=2, label='7-d Rolling Mean', color="b")
ax.plot(quote_trends_30, color="0.2", linewidth=3, label="Trend (30-d Rolling Mean)")

# Major ticks every week.
fmt_week = mdates.WeekdayLocator(interval=1)
ax.xaxis.set_major_locator(fmt_week)

# Minor ticks every day.
fmt_day = mdates.DayLocator(interval=1)
ax.xaxis.set_minor_locator(fmt_day)

# Text in the x axis will be displayed in 'YYYY-mm-dd' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))

ax.legend()
ax.grid(True, linewidth=0.5, linestyle="--")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Quotes")
ax.set_title("Trends in Number of Quotes")

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
fig.autofmt_xdate()

## Number of quote occurrences

TODO: Explore the number of occurrences

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
sns.boxplot(x=dataset["numOccurrences"].astype(np.int64))
plt.show()


Woah! So many outliers... Or are they really outliers?

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
hist, bins = np.histogram(dataset["numOccurrences"].astype(np.int64), bins=16)
log_bins = np.logspace(np.log10(bins[0]), np.log10(bins[-1]), len(bins))
plt.hist(dataset["numOccurrences"].astype(np.int64), bins=log_bins)
plt.xscale('log')
plt.show()

TODO Comment if these are outliers. I believe they are but that is okay -- they might just be very popular or in some way important. Understand which quotes are sooo quoted (print the top ~20).

## Speaker probabilities

TODO: Explore speaker probabilities

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
sns.boxplot(x=dataset["speaker_prob"].astype(np.float64))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.hist(dataset["speaker_prob"].astype(np.float64), bins=25)
plt.show()

## Lenght of the quote

TODO: Explore the length of the quote

In [ ]:
# Get the number of words per quote
dataset["n_word"] = dataset.apply(lambda x: len(re.compile(r'\w+').findall(x["quotation"])), axis=1, meta="np.int64")

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
sns.boxplot(x=dataset["n_word"])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.hist(dataset["n_word"], bins=25)
plt.show()

## Missing values

TODO

## Haiti earthquake case-study

TODO

## Phases

TODO comment a bit on the phases. consult and copy-paste key info from <https://github.com/epfl-dlab/Quotebank/blob/main/phases.md>). show that we are aware of the phases and how we took care of them

## Quote language

TODO check that the quote language is english. Make sure there is no chinese etc. consider dropping non-english quotes because we would prefer to train NLP models that understand only english (for convenience and perhaps feasibility)

## Quote cleaning

TODO Lowercase everything, handle special characters

TODO Not sure if this should be done before EDA or after...

## Newsportal country distribution

We want to understand from which countries the quotes we are wokring with are comming from

TODO visualize number of quotes per country on a world heatmap

TODO not sure which
